In [3]:
-- Load utility scripts (e.g. from from ugcore/scripts)
ug_load_script("ug_util.lua")
ug_load_script("util/refinement_util.lua")


-- Initialize with the world dimension dim=2 and the algebra type
blockSize = 2 -- select 2 for 2x2 point-block system
InitUG(2, AlgebraType("CPU", blockSize));  


In [9]:
-- Parse parameters and print help
gridName= "grids/laplace_sample_grid_2d.ugx" 
numRefs	= 4

-- Load a domain without initial refinements.
mandatorySubsets = {"Inner", "Boundary"}
dom = util.CreateDomain(gridName, 0, mandatorySubsets)

-- Refine the domain (redistribution is handled internally for parallel runs)

util.refinement.CreateRegularHierarchy(dom, numRefs, true)

Loading Domain grids/laplace_sample_grid_2d.ugx ... done.
Performing integrity check on domain ... done.

util.refinement: - refining level 0
util.refinement: - refining level 1
util.refinement: - refining level 2
util.refinement: - refining level 3



## A) Modellparameter
Das Ausgangssystem
$$ \chi (C_m \frac{\partial V}{\partial t} + I)  + \nabla \cdot [ -\sigma \nabla V] = 0$$
transformieren wir zunächst in die dimensionslose Unbekannte $u$:

\begin{align}
u& := (V-V_r)/(V_p-V_r)\\
\Longleftrightarrow V &= (V_p-V_r)*u + V_r
\end{align}
Einheiten:
* [L] = 1 cm
* [T] = 1 ms
* [V] = 1 mV


In [15]:
local VoltPerMilliVolt = 1e-3
local Vr = -85   -- mV
local Vs = -75   -- mV
local Vp =  15   -- mV

local scaleV = (Vp-Vr)        -- Skalierung [mV]
a = (Vs-Vr)/(Vp-Vr)     -- Anregeschwellwert [1]



local Cm  = 1.0 * VoltPerMilliVolt --  uF/cm^2,  where: 1 F=C/V = As / V => 1 mF = ms A/V  => 1 uF = ms * mA /V

c1 = 0.175 -- ms^{-1}
c2 = 0.03  -- ms^{-1}
c3 = 0.011 -- ms^{-1}
b = 0.55  -- [1]


local sigma = 2.0  * VoltPerMilliVolt  -- mS/cm, 1 [S = A/V => 1 mS = 1 mA / V]
local chi = 2000 -- cm^{-1}

DDiff = sigma/(Cm*chi)     --    mA/(V*cm) * V/(mA ms) / cm^{-1} = cm^2 / ms 
local lchar = 1.0 -- characteristic length scale for diffusion
tDiff = lchar*lchar/DDiff


print ("Activation a:"..a)
print ("ScaleV :"..scaleV)


print ("DDiff = "..DDiff.." cm^2/ms")
print ("tDiff = "..tDiff.." ms")
print ("tReact = "..1/c1.." ms")
print ("tReact = "..1/c2.." ms")
print ("tReact = "..1/b.." ms")
print ("tReactDamp = "..1/(a*c1+c3*b).." ms")

uinfty = 0.0
winfty = uinfty / b


Activation a:0.1
ScaleV :100
DDiff = 0.001 cm^2/ms
tDiff = 1000 ms
tReact = 5.7142857142857 ms
tReact = 33.333333333333 ms
tReact = 1.8181818181818 ms
tReactDamp = 42.462845010616 ms


## B) Ansatzraum
Mit den Parametern $a,b,c_1,c_2,c_3, \mathbb D $ lösen wir nun das folgende System:
\begin{align}
\frac{\partial u}{\partial t} + \nabla \cdot [-\mathbb D \nabla u]  &=&  c_1 u (u- a) (1-u)  - c_2 w  \\
\frac{\partial w}{\partial t}  &=& c_3 (u -b w) 
\end{align}
Falls $\mathbb D=0$, so ergeben sich die Gleichgewichte
\begin{align}
u_\infty = 0\\
w_\infty = u_\infty/b = 0
\end{align}

In [17]:
approxSpace = ApproximationSpace(dom)
-- TODO: Add functions

approxSpace:init_levels()
approxSpace:init_top_surface()
approxSpace:print_statistic()

| ---------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                  |
|  Algebra: Block 2 (divide by 2 for #Index)                                   |
|                                                                              |
|    GridLevel   |       Domain |     0: Inner |  1: Boundary                  |
| ---------------------------------------------------------------------------- |
| (lev,    0)    |            0 |            0 |            0 |
| (lev,    1)    |            0 |            0 |            0 |
| (lev,    2)    |            0 |            0 |            0 |
| (lev,    3)    |            0 |            0 |            0 |
| (lev,    4)    |            0 |            0 |            0 |
| (lev,    0, g) |            0 |            0 |            0 |
| (lev,    1, g) |            0 |            0 |            0 |
| (lev,    2, g) |            0 |            0 |            0 |
| 

## C) Elementdiskretisierung (FV)

In [12]:
elemDisc = {}
-- TODO: Define elemDisc["u"], elemDisc["w"]

-- elemDisc["u"]:set_mass_scale(1.0)  -- default
-- elemDisc["w"]:set_mass_scale(1.0)  -- default

-----------------------------------------
-- C).1 Diffusionstensor
-- Df = sigma/(Cm*chi)  
-----------------------------------------
local diffTensorU = DDiff  -- for task a)
-- TODO: Replace 'diffTensorU' by matrix object for task b)

elemDisc["u"]:set_diffusion(diffTensorU)
elemDisc["w"]:set_diffusion(0.0)

-----------------------------------------
-- C).2 Reaktionsterme:
-- \dot u = (c1*u*(u-a)*(1.0-u) - c2*w)
-- \dot w = b*(v - d*w)
-----------------------------------------


function ReactionU(u,w) 
  return -- TODO: implement 
end

function ReactionU_dU(u,w) 
    return -- TODO: implement 
end

function ReactionU_dW(u,w)  
  return -- TODO: implement 
end

function ReactionW(u,w) 
  return -- TODO: implement 
end

function ReactionW_dU(u,w) 
    return -- TODO: implement 
end

function ReactionW_dW(u,w)  
  return -- TODO: implement 
end



local nonlinearGrowth = {}
nonlinearGrowth["u"] = LuaUserFunctionNumber("ReactionU", 2)
nonlinearGrowth["u"]:set_input(0, elemDisc["u"]:value())
nonlinearGrowth["u"]:set_input(1, elemDisc["w"]:value())
nonlinearGrowth["u"]:set_deriv(0, "ReactionU_dU")
nonlinearGrowth["u"]:set_deriv(1, "ReactionU_dW")


-- TODO: nonlinearGrowth["w"] = ...


elemDisc["u"]:set_reaction(nonlinearGrowth["u"])          
elemDisc["w"]:set_reaction(nonlinearGrowth["w"])  

-----------------------------------------
--  C).3 Quellen = Externe Ströme (nicht benutzt!)
-- 1.0 / (Cm*scaleV)
-----------------------------------------
-- function ISourceU(x,y,t,si)  return 0.0/(Cm*scaleV) end
-- elemDisc["u"]:set_source("ISourceU") 

LUA-ERROR: ---

LUA-ERROR! Call stack:
   1  buffer:17        
LUA-ERROR: 
[string "buffer"]:17: attempt to index field 'u' (a nil value)
 % ABORTING script parsing.


## D) Anfangswerte

In [13]:
-- Initial values ("Anfangswerte")
function MyInitialValueU(x, y)
 return -- TODO: implement initial value for "u"
end

function MyInitialValueW(x, y)
  return winfty 
end


## E) Randwerte

In [ ]:
dirichletBND = DirichletBoundary()
dirichletBND:add(uinfty, "u", "Boundary")
dirichletBND:add(winfty, "w", "Boundary")

## F) Diskretisierung auf ganzem Gebiet

In [ ]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc["u"])
domainDisc:add(elemDisc["w"])
domainDisc:add(dirichletBND)


## G) Aufsetzen des Lösers (using 'util/solver_util.lua')

In [ ]:
local solverDesc = {
	
	-- Newton's method for non-linear problem
	type = "newton",

  -- BiCGStab 
	linSolver = {
	 type = "bicgstab",
	
	 precond = {
		  type		= "gmg",
		  approxSpace	= approxSpace,
		  smoother	= "sgs",
		  baseSolver	= "lu"
	 },
	
  },
	
}

nlsolver = util.solver.CreateSolver(solverDesc)
print (nlsolver)

## H) Lösen

In [ ]:
local u = GridFunction(approxSpace)
u:set(0.0)

Interpolate("MyInitialValueU", u, "u")
Interpolate("MyInitialValueW", u, "w")

local startTime = 0
local endTime = tDiff
local dt = (endTime-startTime)/200.0
util.SolveNonlinearTimeProblem(u, domainDisc, nlsolver, VTKOutput(),
"CardioMonoDomain2D", "ImplEuler", 1, startTime, endTime, dt);

print("done")
